# Đồ án Thực Hành Lập Trình Khoa Học Dữ Liệu

In [1]:
import sys
sys.executable

'/home/echkidieu/miniconda3/envs/min_ds-env/bin/python'

<h1 style="color:orange">Import thư viện</h1>

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import datetime as dt
import re

---

<h1 style="color:orange">Thu thập dữ liệu</h1>

- Nhóm sử dụng dữ liệu "steam_games.csv" được thu thập sẵn ở trên kaggle. Dữ liệu này là về các game hiện có ở trên steam. Nguồn của dữ liệu được lấy tại: https://www.kaggle.com/datasets/tristan581/all-55000-games-on-steam-november-2022
- LICENSE của dữ liệu là [CC BY-SA 4.0](https://creativecommons.org/licenses/by-sa/4.0/), người dùng được phép sử dụng với mục đích phi thương mại và phải credit cho chủ sở hữu
- Người ta đã thu thập dữ liệu này bằng cách sử dụng API của steam store và API của web thứ ba (steamspy.com)

---

<h1 style="color:orange">Khám phá dữ liệu</h1>

## Đọc dữ liệu vào dataframe

In [3]:
steam_df = pd.read_csv('steam_games.csv', sep = ';')
steam_df.head()

/home/echkidieu/miniconda3/envs/min_ds-env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3139: DtypeWarning: Columns (19) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,App ID,Name,Short Description,Developer,Publisher,Genre,Tags,Type,Categories,Owners,...,Price,Initial Price,Discount,CCU,Languages,Platforms,Release Date,Required Age,Website,Header Image
0,10,Counter-Strike,Play the world's number 1 online action game. ...,Valve,Valve,Action,"Action: 5426, FPS: 4831, Multiplayer: 3392, Sh...",game,"Multi-player, Valve Anti-Cheat enabled, Online...","10,000,000 .. 20,000,000",...,999,999,0,13990,"English, French, German, Italian, Spanish - Sp...","windows, mac, linux",2000/11/1,0,NaN,https://cdn.akamai.steamstatic.com/steam/apps/...
1,1000000,ASCENXION,ASCENXION is a 2D shoot 'em up game where you ...,IndigoBlue Game Studio,PsychoFlux Entertainment,"Action, Adventure, Indie","""Shoot Em Up"": 186, Metroidvania: 181, Bullet ...",game,"Single-player, Partial Controller Support, Ste...","0 .. 20,000",...,999,999,0,0,"English, Korean, Simplified Chinese",windows,2021/05/14,0,NaN,https://cdn.akamai.steamstatic.com/steam/apps/...
2,1000010,Crown Trick,"Enter a labyrinth that moves as you move, wher...",NEXT Studios,"Team17, NEXT Studios","Adventure, Indie, RPG, Strategy","Rogue-like: 268, Turn-Based Combat: 254, RPG: ...",game,"Single-player, Partial Controller Support, Ste...","200,000 .. 500,000",...,599,1999,70,99,"Simplified Chinese, English, Japanese, Traditi...",windows,2020/10/16,0,NaN,https://cdn.akamai.steamstatic.com/steam/apps/...
3,1000030,"Cook, Serve, Delicious! 3?!","Cook, serve and manage your food truck as you ...",Vertigo Gaming Inc.,Vertigo Gaming Inc.,"Action, Indie, Simulation, Strategy","Typing: 221, Management: 213, Casual: 209, Dif...",game,"Multi-player, Single-player, Co-op, Steam Achi...","100,000 .. 200,000",...,1999,1999,0,76,English,"windows, mac",2020/10/14,0,http://www.cookservedelicious.com,https://cdn.akamai.steamstatic.com/steam/apps/...
4,1000040,细胞战争,这是一款打击感十足的细胞主题游戏！操作简单但活下去却不简单，“你”作为侵入人体的细菌病毒，通...,DoubleC Games,DoubleC Games,"Action, Casual, Indie, Simulation","Action: 22, Casual: 22, Indie: 21, Simulation: 20",game,Single-player,"0 .. 20,000",...,199,199,0,0,Simplified Chinese,windows,2019/03/30,0,NaN,https://cdn.akamai.steamstatic.com/steam/apps/...


## Dữ liệu có bao nhiêu dòng, bao nhiêu cột?

<h4>Tính số dòng và số cột lưu vào hai biến num_rows và num_cols</h4>

In [4]:
num_rows, num_cols = steam_df.shape
num_rows, num_cols

(55691, 22)

## Mỗi dòng có ý nghĩa gì? Có vấn đề các dòng có ý nghĩa khác nhau không?

<h4>Mỗi dòng cho biết thông tin của từng sản phẩm hiện có trong steam, không có dòng nào có ý nghĩa khác với các dòng còn lại.</h4>

## Tiến hành kiểm tra các dòng có bị lặp không

<h4>Kiểm tra bằng cách lưu kết quả vào biến <b>have_duplicated_rows</b>. Biến này có giá trị True nếu dữ liệu có các dòng bị lặp và False nếu ngược lại.</h4>

In [5]:
duplicated_list = steam_df.duplicated()
if True in duplicated_list:
    have_duplicated_rows = True
else:
    have_duplicated_rows = False
have_duplicated_rows

True

<h4>Không có dòng nào bị lặp cả</h4>

## Mỗi cột có ý nghĩa gì?

|STT|Tên cột dữ liệu | <center>Mô tả<center> | Đơn vị |
|--|:------:| ---| --- |
|1|App ID|<center>ID của sản phẩm<center>|
|2|Name| <center>Tên sản phẩm<center>|
|3|Short Description|<center>Phần giới thiệu sản phẩm<center>|
|4|Developer| <center>Nhà phát triển sản phẩm<center>|  |
|5|Publisher|<center>Nhà phân phối sản phẩm<center>|  | 
|6|Genres|<center>Các thể loại của sản phẩm<center>|
|7|Tags|<center>Các tag được gán cho sản phẩm (từ người dùng)<center>|  |
|8|Type|<center>Cho biết sản phẩm thuộc loại game hay là phần cứng (các phần mềm không phải game cũng được gán giá trị là game)<center>| |
|9|Categories|<center>Các hạng mục/đặc trưng của sản phẩm<center>||
|10|Owners|<center>Số lượng người sở hữu sản phẩm<center>| |  
|11|Positive Reviews|<center>Số lượt đánh giá tích cực<center>| |
|12|Negative Reviews|<center>Số lượt đánh giá tiêu cực<center>| |
|13|Price|<center>Giá bán hiện tại của game<center>|USD|
|14|Initial Price|<center>Giá bán của game thời điểm mới phát hành<center>| USD|
|15|Discount|<center>Phần trăm giảm giá của game<center>| \%|
|16|CCU|<center>Số lượng người chơi đồng thời cao nhất (tính vào ngày 8/11/2021)<center>|  |
|17|Languages|<center>Các ngôn ngữ mà sản phẩm có hỗ trợ<center>|  |
|18|Platforms|<center>Các nền tảng mà sản phẩm hỗ trợ<center>||
|19|Release Date|<center>Ngày phát hành sản phẩm<center>|yyyy/mm/dd
|20|Required Age|<center>Độ tuổi giới hạn<center>|
|21|Website|<center>Trang web của nhà phát triển/nhà phân phối<center>|
|22|Header Image|<center>Đường link lưu ảnh của sản phẩm<center>|


<h2>Mỗi cột hiện đang có kiểu dữ liệu gì? Có cột nào có kiểu dữ liệu chưa phù hợp để có thể xử lý tiếp hay không?</h2>

In [6]:
steam_df.columns

Index(['App ID', 'Name', 'Short Description', 'Developer', 'Publisher',
       'Genre', 'Tags', 'Type', 'Categories', 'Owners', 'Positive Reviews',
       'Negative Reviews', 'Price', 'Initial Price', 'Discount', 'CCU',
       'Languages', 'Platforms', 'Release Date', 'Required Age', 'Website',
       'Header Image'],
      dtype='object')

In [7]:
steam_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55691 entries, 0 to 55690
Data columns (total 22 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   App ID             55691 non-null  int64 
 1   Name               55691 non-null  object
 2   Short Description  55654 non-null  object
 3   Developer          55562 non-null  object
 4   Publisher          55540 non-null  object
 5   Genre              55530 non-null  object
 6   Tags               55556 non-null  object
 7   Type               55691 non-null  object
 8   Categories         54721 non-null  object
 9   Owners             55691 non-null  object
 10  Positive Reviews   55691 non-null  int64 
 11  Negative Reviews   55691 non-null  int64 
 12  Price              55691 non-null  int64 
 13  Initial Price      55691 non-null  int64 
 14  Discount           55691 non-null  int64 
 15  CCU                55691 non-null  int64 
 16  Languages          55680 non-null  objec

<h2>Ta thấy một số vấn đề cần giải quyết:</h2>

- Lúc đọc dữ liệu từ "steam_games.csv", ta có thể thấy warning là:"Columns (19) have mixed types.", nghĩa là cột 19(Required Age) có dtype không đồng nhất.<br>
- Owner: thể hiện giá trị khoảng.<br>
- Release Date cần đưa về dạng datetime<br>

<h4>Kiểm tra và xử lý cột <em>Required Age</em> về dạng đồng nhất</h4>

In [8]:
steam_df['Required Age'].unique()

array(['0', '15', '12', '18', '16', '17', '10', '7', '5', '180', '13',
       '3', '6', '14', 'MA 15+', '7+', '21+', 0, 18, 12, 17, 16, 13, 15,
       8, 14, 10, 3, 7, 6, 20, 9, 180, 35], dtype=object)

In [9]:
def open_object_dtype(s):
    dtypes = set(s.apply(type))
    return dtypes
open_object_dtype(steam_df['Required Age'])

{int, str}

- Steam(*https://store.steampowered.com/*) là một nền tảng phân phối trực tuyến, quản lý bản quyền kỹ thuật số, trò chơi điện tử nhiều người chơi, và dịch vụ giao tiếp xã hội trên nền internet phát triển bởi Valve Corporation. Nên các trò chơi điện tử của các quốc gia khác nhau có giới hạn độ tuổi(**Required Age**) khác nhau, sau đây để dễ so sánh và tổng hợp, nhóm mình(em) xin được quy những giới hạn độ tuổi về thang PEGI(*https://pegi.info/what-do-the-labels-mean*).<br>
- Ở cột **Requied Age** ta xử lý chuyển đổi thông tin thành dạng sau đây:<br>
	- 3(PEGI 3) – suitable for all ages, 
	- 7(PEGI 7) – suitable for young children (age>=7)
	- 12(PEGI 12) – suitable for children 12 and over (age>=12)
	- 16(PEGI 16) – suitable for children 16 and over (age>=16)
	- 18(PEGI 18) – Only suitable for adults (age>=18)

In [10]:
def trans_age(type_age):
    l_age=re.findall(r'\d{1,2}',str(type_age))
    age=int(l_age[0])
    if age>=18:
        return 18
    elif age>=16:
        return 16
    elif age>=12:
        return 12
    elif age>=7:
        return 7
    return 3
steam_df['Required Age']=steam_df['Required Age'].apply(trans_age)

<h4>Xử lý cột <em>Owners</em> về các khoảng:</h4>

In [11]:
steam_df.Owners.unique()

array(['10,000,000 .. 20,000,000', '0 .. 20,000', '200,000 .. 500,000',
       '100,000 .. 200,000', '20,000 .. 50,000', '50,000 .. 100,000',
       '500,000 .. 1,000,000', '2,000,000 .. 5,000,000',
       '5,000,000 .. 10,000,000', '1,000,000 .. 2,000,000',
       '20,000,000 .. 50,000,000', '50,000,000 .. 100,000,000',
       '200,000,000 .. 500,000,000'], dtype=object)

In [12]:
def split_owner(st):
    if not(st is np.nan):
        st=st.replace(',','')
        value=st.split('..')
        return tuple([int(value[0]),int(value[1])])
    return np.nan
steam_df.Owners=steam_df.Owners.apply(split_owner)

<h4>Xử lý cột <em>Release Date</em> về đúng dạng chuẩn yyyy/mm/dd</h4>

In [13]:
steam_df['Release Date']=steam_df['Release Date'].astype('datetime64[s]')

In [14]:
steam_df.head()

,App ID,Name,Short Description,Developer,Publisher,Genre,Tags,Type,Categories,Owners,...,Price,Initial Price,Discount,CCU,Languages,Platforms,Release Date,Required Age,Website,Header Image
0,10,Counter-Strike,Play the world's number 1 online action game. ...,Valve,Valve,Action,"Action: 5426, FPS: 4831, Multiplayer: 3392, Sh...",game,"Multi-player, Valve Anti-Cheat enabled, Online...","(10000000, 20000000)",...,999,999,0,13990,"English, French, German, Italian, Spanish - Sp...","windows, mac, linux",2000-11-01,3,NaN,https://cdn.akamai.steamstatic.com/steam/apps/...
1,1000000,ASCENXION,ASCENXION is a 2D shoot 'em up game where you ...,IndigoBlue Game Studio,PsychoFlux Entertainment,"Action, Adventure, Indie","""Shoot Em Up"": 186, Metroidvania: 181, Bullet ...",game,"Single-player, Partial Controller Support, Ste...","(0, 20000)",...,999,999,0,0,"English, Korean, Simplified Chinese",windows,2021-05-14,3,NaN,https://cdn.akamai.steamstatic.com/steam/apps/...
2,1000010,Crown Trick,"Enter a labyrinth that moves as you move, wher...",NEXT Studios,"Team17, NEXT Studios","Adventure, Indie, RPG, Strategy","Rogue-like: 268, Turn-Based Combat: 254, RPG: ...",game,"Single-player, Partial Controller Support, Ste...","(200000, 500000)",...,599,1999,70,99,"Simplified Chinese, English, Japanese, Traditi...",windows,2020-10-16,3,NaN,https://cdn.akamai.steamstatic.com/steam/apps/...
3,1000030,"Cook, Serve, Delicious! 3?!","Cook, serve and manage your food truck as you ...",Vertigo Gaming Inc.,Vertigo Gaming Inc.,"Action, Indie, Simulation, Strategy","Typing: 221, Management: 213, Casual: 209, Dif...",game,"Multi-player, Single-player, Co-op, Steam Achi...","(100000, 200000)",...,1999,1999,0,76,English,"windows, mac",2020-10-14,3,http://www.cookservedelicious.com,https://cdn.akamai.steamstatic.com/steam/apps/...
4,1000040,细胞战争,这是一款打击感十足的细胞主题游戏！操作简单但活下去却不简单，“你”作为侵入人体的细菌病毒，通...,DoubleC Games,DoubleC Games,"Action, Casual, Indie, Simulation","Action: 22, Casual: 22, Indie: 21, Simulation: 20",game,Single-player,"(0, 20000)",...,199,199,0,0,Simplified Chinese,windows,2019-03-30,3,NaN,https://cdn.akamai.steamstatic.com/steam/apps/...


## Phân bố dữ liệu ở các cột:

<h3>Với các cột có kiểu dữ liệu dạng số:</h3>

Với các cột có kiểu dữ liệu số, ta tính:

- Tỉ lệ % các giá trị thiếu
- Giá trị min
- Giá trị max

In [15]:
numeric_cols=steam_df[steam_df.select_dtypes(include=[int,'datetime64[ns]']).columns]

def missing_ratio(col):
    return np.round(col.isna().sum()/len(col)*100,4)
def min(col):
    return col.min()
def max(col):
    return col.max()

num_col_profiles_df=numeric_cols.agg([missing_ratio,min,max])
num_col_profiles_df



,App ID,Positive Reviews,Negative Reviews,Price,Initial Price,Discount,CCU,Release Date,Required Age
missing_ratio,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1778,0.0
min,10.0,0.0,0.0,0.0,0.0,0.0,0.0,1997-06-30 00:00:00,3.0
max,2190950.0,5943345.0,908515.0,99900.0,99900.0,90.0,874053.0,2022-11-11 00:00:00,18.0


<h3>Với các cột có kiểu dữ liệu không phải là dạng số:</h3>

Với các dữ liệu không phải số:

- Ta thấy các cột chưa đúng dạng chuẩn *(Bị thừa các ký tự không cần thiết, nội dung giống nhau nhưng sắp xếp khác nhau)*.
- Với các cột **Name, Short Description,Website, Header Image** là những cột chứa thông tin đặc trưng của từng game, nên giá trị phân bố là trải rộng đều với nhau.
- Với cột **Tags** chứa thông tin của từng Genre của game *(người dùng tag)* nên ở đây ta cũng không xét phân bố như thế nào

In [16]:
object_cols=steam_df.loc[:,['Developer', 'Publisher', 'Genre', 'Type','Categories', 'Owners', 'Languages', 'Platforms']]
def split_str(st):
    if not (st is np.nan):
        return st.split(',')
    return []
num_value={}
ratio_values={}
obj_col_profiles_df=object_cols.agg([missing_ratio])

<h3>Developer</h3>

In [17]:
def lower_str(st):
    if not (st is np.nan):
        return st.lower()
    return ""
def convert_co_inc(st):
    if not (st is np.nan):
        st = re.sub(" *ltd.?[ .]*","ltd.,", st)
        st= re.sub(" *inc.?[ .]*","inc.,", st)
        st= re.sub(" *llc.?[ .]*","llc.,", st)
        return re.sub(r" *, *(?!inc|ltd|llc)",";", st)
    return ""
def split_dev(st):
    if not (st is np.nan):
        return st.split(';')
    return []

ratio_value=np.round(steam_df.Developer.apply(lower_str).apply(convert_co_inc).apply(split_dev).explode().replace("",np.nan).value_counts(normalize=True).multiply(100))
num_value.update({"Developer":len(ratio_value)})
ratio_values.update({"Developer":str(ratio_value.to_dict())})

<h3>Publisher</h3>

In [18]:
ratio_value=np.round(steam_df.Publisher.apply(lower_str).apply(convert_co_inc).apply(split_dev).explode().replace("",np.nan).value_counts(normalize=True).multiply(100))
num_value.update({"Publisher":len(ratio_value)})
ratio_values.update({"Publisher":str(ratio_value.to_dict())})

<h3>Genre</h3>

In [19]:
col=steam_df.Genre.apply(split_str).explode().str.strip()
ratio_value=np.round(col.value_counts(normalize=True).multiply(100))
num_value.update({"Genre":len(ratio_value)})
ratio_values.update({"Genre":str(ratio_value.to_dict())})

<h3>Type</h3>

In [20]:
ratio_value=np.round(steam_df.Type.value_counts(normalize=True).multiply(100))
num_value.update({"Type":len(ratio_value)})
ratio_values.update({"Type":str(ratio_value.to_dict())})

<h3>Owners</h3>

In [21]:
ratio_value=np.round(steam_df.Owners.value_counts(normalize=True).multiply(100))
num_value.update({"Owners":len(ratio_value)})
ratio_values.update({"Owners":str(ratio_value.to_dict())})

<h3>Categories</h3>

In [22]:
col=steam_df.Categories.apply(split_str).explode().str.strip()
ratio_value=np.round(col.value_counts(normalize=True).multiply(100))
num_value.update({"Categories":len(ratio_value)})
ratio_values.update({"Categories":str(ratio_value.to_dict())})

<h3>Languages</h3>

In [23]:
def clean_lang(lang_str):
    if not (lang_str is np.nan):
        return lang_str.replace('\r\n',',').replace('[b]*[/b]','').replace(';','').replace('#lang_slovakian', 'Slovak')
    return ""

col=steam_df.Languages
# Xử lý dữ liệu
col=col.apply(clean_lang).apply(split_str).explode().str.strip()
col=col.apply(lambda st: st.split('-')[0].strip()).replace('',np.nan)

# Xem các giá trị trong cột được phân bố như thế nào
ratio_value=np.round(col.value_counts(normalize=True).multiply(100),4)
num_value.update({"Languages":len(ratio_value)})
ratio_values.update({"Languages":str(ratio_value.to_dict())})

<h3>Platforms</h3>

In [24]:
col = steam_df['Platforms']
col=col.apply(split_str).explode().str.strip()
ratio_value=np.round(col.value_counts(normalize=True).multiply(100),4)
num_value.update({"Platforms":len(ratio_value)})
ratio_values.update({"Platforms":str(ratio_value.to_dict())})

In [25]:
obj_col_profiles_df=obj_col_profiles_df.append(pd.DataFrame(num_value, index=[0]))
obj_col_profiles_df.rename(index={0:'num_value'},inplace=True)
obj_col_profiles_df=obj_col_profiles_df.append(pd.DataFrame(ratio_values,index=[0]))
obj_col_profiles_df.rename(index={0:'ratio_value'},inplace=True)

<h4>Bảng phân bố các dữ liệu <b>không</b> là dữ liệu số</h4>

In [26]:
pd.set_option('display.max_colwidth',200)
obj_col_profiles_df

,Developer,Publisher,Genre,Type,Categories,Owners,Languages,Platforms
missing_ratio,0.2316,0.2711,0.2891,0.0,1.7418,0.0,0.0198,0.0
num_value,36550.0,29925.0,28.0,2.0,36.0,13.0,75.0,3.0
ratio_value,"{'choice of games': 0.0, 'creobit': 0.0, 'laush dmitriy sergeevich': 0.0, 'sokpop collective': 0.0, 'koei tecmo games co.,ltd.': 0.0, 'reforged group': 0.0, 'dnovel': 0.0, 'boogygames studios': 0....","{'big fish games': 1.0, '8floor': 0.0, 'sega': 0.0, 'strategy first': 0.0, 'square enix': 0.0, 'thq nordic': 0.0, 'choice of games': 0.0, 'plug in digital': 0.0, 'sekai project': 0.0, 'hh-games': ...","{'Indie': 25.0, 'Action': 15.0, 'Casual': 14.0, 'Adventure': 14.0, 'Strategy': 7.0, 'Simulation': 7.0, 'RPG': 6.0, 'Early Access': 4.0, 'Free to Play': 2.0, 'Sports': 2.0, 'Racing': 1.0, 'Massivel...","{'game': 100.0, 'hardware': 0.0}","{'Single-player': 27.0, 'Steam Achievements': 14.0, 'Steam Cloud': 7.0, 'Full controller support': 6.0, 'Multi-player': 6.0, 'Steam Trading Cards': 5.0, 'Partial Controller Support': 4.0, 'PvP': 4...","{(0, 20000): 68.0, (20000, 50000): 13.0, (50000, 100000): 7.0, (100000, 200000): 5.0, (200000, 500000): 4.0, (500000, 1000000): 2.0, (1000000, 2000000): 1.0, (2000000, 5000000): 1.0, (5000000, 100...","{'English': 27.2476, 'Spanish': 7.3987, 'German': 6.932, 'French': 6.6374, 'Russian': 6.3883, 'Simplified Chinese': 6.3186, 'Portuguese': 5.3249, 'Japanese': 5.1257, 'Italian': 4.6003, 'Korean': 3...","{'windows': 72.3968, 'mac': 16.6051, 'linux': 10.9981}"


## Đưa ra các câu hỏi

YOUR TEXT HERE

## Tiền xử lý và phân tích dữ liệu để trả lời cho từng câu hỏi

Studio nào đa dạng về thể loại game nhất?

Mối quan hệ giữa giá bán, giá khởi điểm và số lượng người tải

Thể loại có lượt người tag nhiều nhất. Tương quan giữa lượt tag thể loại của game đó với thể loại chính thức của game (liệu có game nào có một thể loại được nhiều người tag nhưng lại không có trong thể loại chính thức không)